## Ejercicio 4
### Regresión Logistica

### Churn: 
Pérdida/deserción de clientes de la compañía. Especialmente, las industrias en las que la adquisición de usuarios es costosa, es de vital importancia para una empresa reducir e idealmente hacer que la deserción se reduzca a 0 para mantener sus ingresos recurrentes. Si se considera que la retención de clientes es siempre más barata que la adquisición de clientes nuevos, se plantea un problema emocionante/difícil para ML.

### Data
El dataset proviene de un proveedor de servicios de telecomunicaciones donde tienen el uso del servicio (plan internacional, plan de correo de voz, uso durante el día, uso en tardes y noches, etc.) e información demográfica básica (estado y código de área) del usuario. Y es que el cliente esté retirado o no.

In [63]:
import pandas as pd
data = pd.read_csv('https://github.com/vanejaime/Curso_Python_CD/raw/master/datasets/churn.csv')

In [64]:
data.head()

,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [65]:
data.columns

Index(['Account Length', 'Area Code', 'Phone', 'Int'l Plan', 'VMail Plan',
       'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins',
       'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge',
       'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?'],
      dtype='object')

# Ejercicio 4.1

Cree Y y X

Cual es la distribución de los churners?

Divida el dataset en train (70%) and test (30%)

In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Distribución Churn.
data['Resp'] = np.where(data['Churn?'] =='False.', 0, 1)
print(data['Resp'].value_counts(normalize = False, dropna = False))
print(data['Resp'].value_counts(normalize = True, dropna = False))
sns.set(style="darkgrid")
sns.countplot(x = 'Resp', data = data)
plt.show()
data.drop(['Churn?'], axis = 1)

# Exploración de la Data.
# print(data.groupby('Resp').mean())

# Variables Categoricas.
cat_vars = ["Int'l Plan",'VMail Plan']
for var in cat_vars:
    cat_list = 'var' + '_' + var
    cat_list = pd.get_dummies(data[var], prefix = var)
    data1 = data.join(cat_list)
    data = data1
data_vars = data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
data_final=data[to_keep]
data_final.columns.values
data_final.head()

# Creación de bases para entrenar y probar.
X = data_final.loc[:, data_final.columns != 'Resp']
y = data_final.loc[:, data_final.columns == 'Resp']

#try:
#    import imbalanced-learn
#except:
#    import pip
#    pip.main(['install','imbalanced-learn'])
import 'imbalanced-learn'
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
columns = X_train.columns
os_data_X,os_data_y = os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data = os_data_X,columns = columns)
os_data_y= pd.DataFrame(data = os_data_y,columns = ['Resp'])

print("length of oversampled data is ",len(os_data_X))
print("Number of no 'Retirados' in oversampled data",len(os_data_y[os_data_y['Resp']==0]))
print("Number of retirados",len(os_data_y[os_data_y['Resp']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['Resp']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['Resp']==1])/len(os_data_X))

data_final_vars=data_final.columns.values.tolist()
y=['Resp']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

#Las variables que mas explican
cols=[] 
X=os_data_X[cols]
y=os_data_y['Resp']

import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

# Creación de bases para entrenar y probar.
# print(data[:].shape)

#X_train, X_test, y_train, y_test = train_test_split(data, data['Churn?'], test_size=0.3)
#print(X_train.shape, y_train.shape)
#print(X_test.shape, y_test.shape)

# Correr el modelo.
#logmodel = LogisticRegression()
#logmodel.fit(X_train, y_train)

# Predicciones del modelo.
#Predictions = logmodel.predict(X_test)
#print(classification_report(y_test,predictions))
#print(confusion_matrix(y_test, predictions))


IndentationError: unexpected indent (<ipython-input-66-e7a9b9074e61>, line 43)

# Ejercicio 4.2

Entrene una regresion logistica usando el set de train y aplique el algoritmo en el set de test

# Ejercicio 4.3

a) Cree la matriz de confusion 

b) Estime el accuracy y otras medidas desempeño